In [100]:
import math
import numpy as np
import matplotlib.pyplot as plt
from docplex.mp.basic import Expr
from docplex.mp.model import Model
from docplex.mp.solution import SolveSolution
from docplex.util.status import JobSolveStatus
import cplex.callbacks as cpx_cb
from Functions_validation.braking_poly import p as Pbraking
from Functions_validation.traction_poly import p as Ptraction
from Functions_validation.RAV_poly import P as Prav

from Functions_validation.braking_poly import Pdecc as Pdecc
from Functions_validation.traction_poly import Pacc as Pacc
from Functions_validation.power_poly import P as Ppower

In [101]:
A, B, C, m, v_max, p_max, c_max = 0.632, 40.7, 3900, 320000, 220, 4305220, 320000
import numpy as np


def P(v):
    return A + B * v + C * v**2


a, b = 0, 250
n = 1

# Choose n+1 interpolation points
x_points = np.linspace(a, b, 1000)
y_points_bis = []
for x in x_points:
    y_points_bis.append(P(x))

# Get the polynomial coefficients (highest degree first)
coeffs = np.polyfit(x_points, y_points_bis, deg=n)

# Create polynomial function
Prav = np.poly1d(coeffs)


def Prav_approx(v):
    return Prav(v)

In [102]:
# Define some constants
A, B, C, m, v_max, p_max, c_max = 0.632, 40.7, 3900, 320000, 220, 4305220, 320000


## Defining the Model

### Define model

In [103]:
opt_model = Model(name="Poly_Model")
N = 20
Dist = 30
tolerance = 0.005

### Define variables

In [104]:
v={}
for i in range(0, N):
    v[i]= opt_model.continuous_var(name=f"v_{i}")

d={}
for i in range(0, N):
    d[i]= opt_model.continuous_var(name=f"d_{i}")

kroe={}
for i in range(0, N):
    kroe[i]= opt_model.binary_var(name=f"kroe_{i}")

### Define cost

In [105]:
# Total cost
total_cost = opt_model.sum(Ppower(v[i], d[i]) for i in range(N))

# Set objective
opt_model.minimize(total_cost)

### Kroe constraints

In [65]:
M = 1
epsilon = 1e-4

# Constraint: if delta == 1, then d >= epsilon
for i in range(0, N):
    opt_model.add_constraint(d[i] >= epsilon - M * (1 - kroe[i]))

    # Constraint: if delta == 0, then d <= 0
    opt_model.add_constraint(d[i] <= M * kroe[i])

### Speed constraints

In [106]:
opt_model.add_constraint(v[0] == 0, "Initial velocity")
opt_model.add_constraint(v[N-1] == 0, "Final velocity")

for i in range(1,N-2):
    opt_model.add_constraint(
    v[i+1] - v[i] + (1/m)*Prav(v[i]) == (1/m) * (d[i] * (kroe[i] * 123777.38520754 + (1 - kroe[i]) * 134886.16359324)),
    f"Velocity constraint {i}"
)
    '''opt_model.add_constraint(v[i+1] - v[i] + (1/m)*Prav(v[i]) == (1/m)(d[i]*(kroe[i]*Ptraction(v[i]) + (1-kroe[i])*Pbraking(v[i]))), f"Velocity constraint {i}")'''

In [107]:
opt_model.print_information() 

print(opt_model.prettyprint())    

Model: Poly_Model
 - number of variables: 60
   - binary=20, integer=0, continuous=40
 - number of constraints: 19
   - linear=2, quadratic=17
 - parameters: defaults
 - objective: minimize quadratic
 - problem type is: MIQCP
// This file has been generated by DOcplex
// model name is: Poly_Model
// single vars section
dvar float v_0;
dvar float v_1;
dvar float v_2;
dvar float v_3;
dvar float v_4;
dvar float v_5;
dvar float v_6;
dvar float v_7;
dvar float v_8;
dvar float v_9;
dvar float v_10;
dvar float v_11;
dvar float v_12;
dvar float v_13;
dvar float v_14;
dvar float v_15;
dvar float v_16;
dvar float v_17;
dvar float v_18;
dvar float v_19;
dvar float d_0;
dvar float d_1;
dvar float d_2;
dvar float d_3;
dvar float d_4;
dvar float d_5;
dvar float d_6;
dvar float d_7;
dvar float d_8;
dvar float d_9;
dvar float d_10;
dvar float d_11;
dvar float d_12;
dvar float d_13;
dvar float d_14;
dvar float d_15;
dvar float d_16;
dvar float d_17;
dvar float d_18;
dvar float d_19;
dvar bool kroe_0;
d

In [108]:
result = opt_model.solve(log_output=True) #(log_output=self.solver_config.cplex_log)

Version identifier: 22.1.2.0 | 2024-12-10 | f4cec290b
CPXPARAM_Read_DataCheck                          1
CPLEX Error  5002: 'Velocity constraint 1' is not convex.
Presolve time = 0.00 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.01 ticks)
Parallel b&c, 20 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.01 ticks)
Error: Model has non-convex quadratic constraint, name='Velocity constraint 1'
